In [ ]:
from pathlib import Path

import matplotlib.pyplot as plt

from mxlpy import Model, fns, mc, plot
from mxlpy.distributions import GaussianKde, sample
from mxlpy.parameterise import get_km_and_kcat_from_brenda

# Model parameterisation

Obtaining experimentally measured parameters can be challenging.  

Using the [Brenda enzymes database](https://www.brenda-enzymes.org/) we can obtain  distributions of enzymatic parameters for a wide range of organisms.

We can do that with the `mxlpy.parameterise` module.  

These distributions can then in turn be used with our [Monte-Carlo methods](monte-carlo.ipynb) to capture the **range of possible behaviour** your model can exhibit.

<div>
    <img src="assets/time-course.png"
         style='vertical-align:middle; max-height: 175px; max-width: 20%;'/>
    <span style='padding: 0 1rem; font-size: 2rem'>+</span>
    <img src="assets/parameter-distribution.png"
         style='vertical-align:middle; max-height: 175px; max-width: 20%'/>
    <span style='padding: 0 1rem; font-size: 2rem'>=</span>
    <img src="assets/mc-time-course.png"
         style='vertical-align:middle; max-height: 175px; max-width: 20%'/>
</div>


In order to obtain the parameters for a given [Enzyme commision number](https://en.wikipedia.org/wiki/Enzyme_Commission_number) (ec) we will [manually download the database](https://www.brenda-enzymes.org/download.php).  
You have to do this manually due to the brenda licensing terms.

> Note: we have created a small copy of just the rubisco data here to keep the documentation running.  
> Adjust your `brenda_path` accordingly

In [ ]:
kms, kcats = get_km_and_kcat_from_brenda(
    ec="4.1.1.39",
    brenda_path=Path("assets") / "brenda_rubisco_only.json",
)

print(f"Found: {len(kms)} michaelis constants")
kms.head()

As you can see above, this provides you with parameter values for different organisms and substrates.  
Thus, we first filter by the specific substrate we are interested in.  

In [ ]:
# Filter out a specific substrate
kms = kms[kms["substrate"] == "CO2"]
kcats = kcats[kcats["substrate"] == "CO2"]

print(f"Filtered to {len(kms)} michaelis constants")
kms.head()

Since these are sufficiently many values, we can create a Gaussian Kernel Density estimate of them.

In [ ]:
km_dist = GaussianKde.from_data(kms["value"])
fig, ax = km_dist.plot(
    xmin=kms["value"].min() * 0.8,
    xmax=kms["value"].max() * 1.2,
)
ax.set(title=f"rubisco km for CO2, n={len(kms)}")
plt.show()

This kernel density estimate we can now use exactly like other distribution in our `Monte-Carlo` routines (see the Monte Carlo notebook for more information).  

Here, we create a small toy model and then use the distribution obtained from the experimental data to calculate the steady-state distribution of the model concentration.  

In [ ]:
model = (
    Model()
    .add_parameters({"k_out": 1.0, "km": 1.0})
    .add_variable("PGA", 0)
    .add_reaction(
        "rubisco",
        fns.constant,
        args=["km"],
        stoichiometry={"PGA": 2},
    )
    .add_reaction(
        "outflux",
        fns.mass_action_1s,
        args=["PGA", "k_out"],
        stoichiometry={"PGA": -1},
    )
)

ss = mc.steady_state(model, mc_to_scan=sample({"km": km_dist}, n=10))

fig, ax = plt.subplots(figsize=(4, 3))
ax.set(ylabel="Steady-state concentration")
plot.violins(ss.variables, ax=ax)
plt.show()

<div style="color: #ffffff; background-color: #04AA6D; padding: 3rem 1rem 3rem 1rem; box-sizing: border-box">
    <h2>First finish line</h2>
    With that you now know most of what you will need from a day-to-day basis about parameter scans in mxlpy.
    <br />
    <br />
    Congratulations!
</div>